本段尽可能通过调用api的方式进行。

In [ ]:
import pyaudio,wave
import ollama
import warnings
import os
import glob
import sys
import torch
import re
import logging
import pickle
os.environ['HF_ENDPOINT']='hf-mirror.com'#hugging_face镜像
# os.environ['HF_HUB_OFFLINE']='0'
#清理内存
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
if hasattr(torch.cuda, 'empty_cache'):
    torch.cuda.empty_cache()
del torch
warnings.filterwarnings("ignore")

In [ ]:
#语音合成：GPT_Sovits---fast_inference version
sys.path.insert(0,'../Speech_Synthesis\GPT_Sovits\GPT-SoVITS-beta\GPT-SoVITS-beta_fast_inference_0316\GPT_SoVITS')
from inference_webui_fast_inference_maple import handle

#ASR模型：sherpa_onnx---cpu版本
import sherpa_onnx
path = 'E:/LargeModel/kaldi/sherpa_onnx_model/stream_zipformer'
sys.path.append(path)
import microphone_endpoint
args_dict = {  
    'tokens': path+'/tokens.txt',  
    'encoder': path+'/encoder-epoch-99-avg-1.onnx',  
    'decoder': path+'/decoder-epoch-99-avg-1.onnx' ,  
    'joiner': path+'/joiner-epoch-99-avg-1.onnx',    
} 
#检查设备，顺便输出recongnizer
check_recognizer= microphone_endpoint.main(args_dict)

#文本模型：chatglm
from zhipuai import ZhipuAI
with open(r"E:\LargeModel\Language_Model\Text_generation\chatglm_api\api_key.txt", 'r') as chatglm_api:
    api_key = chatglm_api.read()
client = ZhipuAI(api_key=api_key.split('=')[1].strip('"')) # 填写您自己的APIKey

In [ ]:
#语音合成模型初始化
ref_wav_path="E:\LargeModel\Speech_Synthesis\GPT_Sovits\GPT-SoVITS-beta\派蒙--官方给的\参考音频\说话—既然罗莎莉亚说足迹上有元素力，用元素视野应该能很清楚地看到吧。.wav"
prompt_text= '既然罗莎莉亚说足迹上有元素力，用元素视野应该能很清楚地看到吧。'
text='晚上好，祝你有一个甜美的睡眠！'#'你好呀！'
top_k=5
top_p=1
temperature=1
how_to_cut='按标点符号切'
sovits_path = r"E:\LargeModel\Speech_Synthesis\GPT_Sovits\GPT-SoVITS-beta\GPT-SoVITS-beta_fast_inference_0316\GPT_SoVITS\pretrained_models\s2G488k.pth"
gpt_path = r"E:\LargeModel\Speech_Synthesis\GPT_Sovits\GPT-SoVITS-beta\GPT-SoVITS-beta_fast_inference_0316\GPT_SoVITS\pretrained_models\s1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt"
audio_save=False
audio_save_file='output.wav'
text_language2='中英混合'
prompt_language2='中英混合'
batch_size=6         # int. batch size for inference
split_bucket=True   # bool. whether to split the batch into multiple buckets.
speed_factor=1.0
ref_text_free=False

params={'text':text, 'text_lang':text_language2, 
            'ref_audio_path':ref_wav_path, 'prompt_text':prompt_text, 
            'prompt_lang':prompt_language2, 'top_k':top_k, 
            'top_p':top_p, 'temperature':temperature, 
            'text_split_method':how_to_cut, 'batch_size':batch_size, 
            'speed_factor':speed_factor, 'ref_text_free':ref_text_free,
            'split_bucket':split_bucket, 'audio_save':audio_save, 
            'audio_save_file':audio_save_file, 'mygpt_path':gpt_path, 'mysovits_path':sovits_path}

def tts(params):
    # 将标准输出流重定向到空设备
    original_stdout =  sys.stdout
    sys.stdout = open('log_maple.txt', 'w')
    handle(**params)
    sys.stdout = original_stdout
    logging.getLogger().setLevel(logging.WARNING)

tts(params)

In [ ]:
#拜拜结束对话
#对话前删除临时录音文件
def delete_files(folder_path):
    file_list = glob.glob(folder_path + '/*')
    for file_path in file_list:
        if os.path.isfile(file_path):
            if '.' in file_path and file_path.split('.')[0].split('/')[-1][:4]=='test':
                os.remove(file_path)

# 调用函数删除文件夹中的文件
audio_file_path = 'test'
delete_files(audio_file_path)

#语音提醒可以对话了
params['text'] ="您好，您现在可以跟我进行对话了！" 
tts(params)

outfile_name=audio_file_path+'\\test.mp3'
lc =0 #对话轮次，用于录音文件的名称存储
messages = [{"role": "system", "content":"您是一个知心朋友，你的任务是和用户聊天。"}]
tokens_use = 0
while(True):
    #ASR
    last_result, lc_result = microphone_endpoint.main(args_dict, False, check_recognizer)
    #结束语
    if re.sub(r'[ .。！!?？-]','', last_result).lower() in {'拜拜','byebye','再见','掰掰'}:
        lc=-1
        result=last_result
    
    if len(lc_result)==0:
        print('\n', '*'*40)
        print(' '*5, '您的对话已超时，请重新对话！')
        print('\n', '*'*40)
        break

    print()
    #每轮的文本互动开始
    # print('Bot:',end='')
    if lc==-1:
        print(result)
        break
    lc +=1
    #--------------------Text to Text
    #剪切历史记录每9条删除前面4条
    if len(messages)>7:
        for i in range(4):
            messages.pop(1)
    
    question = lc_result
    if (len(question)==0)|(question=='拜拜'):
        break
    print('智谱：', end='')
    messages.append({"role": "user", "content": question})
    model = 'glm-4'
    response = client.chat.completions.create(
        model=model,  # 填写需要调用的模型名称
        messages=messages,
        # max_tokens=1024,#所以历史记录加起来不能超过这个长度
        stream=True,
    )
    answer=''
    outword=''
    for chunk in response:
        answer_chunk = chunk.choices[0].delta.content
        outword += answer_chunk
        answer+= answer_chunk
        print(answer_chunk, end='', flush=True)
        if answer_chunk in {"\n"}:
            params['text']=outword
            tts(params)
            outword=''
    if len(outword)!=0:
        params['text']=outword
        tts(params)
    print('####end', end='')
    params['text'] ='####end'
    tts(params)
    print()
    tokens_use += chunk.usage.total_tokens
    messages.append({"role": "assistant", "content": answer})

In [ ]:
#存储聊天记录
with open(r'E:\LargeModel\Language_Model\Text_generation\chatglm_api\chat_message.pkl', 'wb') as save_file:
    pickle.dump(messages, save_file)
tokens_use